## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
val_count = application_df['APPLICATION_TYPE'].value_counts()
val_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(val_count[val_count<200].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
class_val_count = application_df['CLASSIFICATION'].value_counts()
class_val_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1820        1
C4200        1
C3700        1
C2600        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_val_count2 = class_val_count[class_val_count>1]
class_val_count2

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1600        5
C1257        5
C0           3
C2710        3
C1260        3
C3200        2
C1234        2
C1267        2
C1256        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace =class_val_count.loc[class_val_count<1000].index

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(application_df)
dummies_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_Other,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = dummies_df.drop(['IS_SUCCESSFUL'],1).values
y = dummies_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state= 42)

In [12]:
print(X.shape)
print(y.shape)

(34299, 43)
(34299,)


In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 80
hidden_nodes_layer2 = 30
#hidden_nodes_layer3 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 4ms/step - loss: 0.5712 - accuracy: 0.7208
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5569 - accuracy: 0.7266
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5536 - accuracy: 0.7293
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5510 - accuracy: 0.7303
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5499 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5478 - accuracy: 0.7309
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5474 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5469 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5458 - accura

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5639 - accuracy: 0.7277 - 735ms/epoch - 3ms/step
Loss: 0.5639024972915649, Accuracy: 0.7276967763900757


In [18]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5") 

**Optimization 1**


In [19]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 43
hidden_nodes_layer2 = 25
#hidden_nodes_layer3 = 15

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))


# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 43)                1892      
                                                                 
 dense_4 (Dense)             (None, 25)                1100      
                                                                 
 dense_5 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,018
Trainable params: 3,018
Non-trainable params: 0
_________________________________________________________________


In [20]:
# Compile the model
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [21]:
# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5765 - accuracy: 0.7133
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5556 - accuracy: 0.7291
Epoch 3/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5529 - accuracy: 0.7300
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5502 - accuracy: 0.7303
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7304
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5483 - accuracy: 0.7320
Epoch 7/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7312
Epoch 8/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7320
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7326
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accura

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5651 - accuracy: 0.7263 - 711ms/epoch - 3ms/step
Loss: 0.5651004910469055, Accuracy: 0.7262973785400391


In [23]:
# Export our model to HDF5 file
nn2.save("AlphabetSoupCharity_optimize_1.h5") 

**Optimization 2**

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 20
hidden_nodes_layer2 = 40
hidden_nodes_layer3 = 80

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))


# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 20)                880       
                                                                 
 dense_7 (Dense)             (None, 40)                840       
                                                                 
 dense_8 (Dense)             (None, 80)                3280      
                                                                 
 dense_9 (Dense)             (None, 1)                 81        
                                                                 
Total params: 5,081
Trainable params: 5,081
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5725 - accuracy: 0.7208
Epoch 2/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5562 - accuracy: 0.7286
Epoch 3/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5533 - accuracy: 0.7282
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5513 - accuracy: 0.7301
Epoch 5/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5496 - accuracy: 0.7310
Epoch 6/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5487 - accuracy: 0.7313
Epoch 7/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5481 - accuracy: 0.7322
Epoch 8/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5475 - accuracy: 0.7318
Epoch 9/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7338
Epoch 10/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5458 - accura

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5761 - accuracy: 0.7322 - 742ms/epoch - 3ms/step
Loss: 0.576053261756897, Accuracy: 0.7322449088096619


In [28]:
# Export our model to HDF5 file
nn3.save("AlphabetSoupCharity_optimize_2.h5") 

In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 60
hidden_nodes_layer2 = 60
hidden_nodes_layer3 = 40
hidden_nodes_layer4 = 20

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=input_features, activation='relu'))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Forth hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation='relu'))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 60)                2640      
                                                                 
 dense_11 (Dense)            (None, 60)                3660      
                                                                 
 dense_12 (Dense)            (None, 40)                2440      
                                                                 
 dense_13 (Dense)            (None, 20)                820       
                                                                 
 dense_14 (Dense)            (None, 1)                 21        
                                                                 
Total params: 9,581
Trainable params: 9,581
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn4.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model
fit_model = nn4.fit(X_train_scaled, y_train, epochs=200)

Epoch 1/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5713 - accuracy: 0.7197
Epoch 2/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5558 - accuracy: 0.7264
Epoch 3/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5527 - accuracy: 0.7309
Epoch 4/200
804/804 [==============================] - 3s 4ms/step - loss: 0.5503 - accuracy: 0.7327
Epoch 5/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5492 - accuracy: 0.7311
Epoch 6/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5477 - accuracy: 0.7318
Epoch 7/200
804/804 [==============================] - 4s 5ms/step - loss: 0.5467 - accuracy: 0.7329
Epoch 8/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5464 - accuracy: 0.7341
Epoch 9/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5451 - accuracy: 0.7333
Epoch 10/200
804/804 [==============================] - 4s 4ms/step - loss: 0.5449 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6118 - accuracy: 0.7266 - 769ms/epoch - 3ms/step
Loss: 0.6118495464324951, Accuracy: 0.7266472578048706


In [33]:
# Export our model to HDF5 file
nn4.save("AlphabetSoupCharity_optimize_3.h5") 